In [ ]:
import islpy as isl

import time




In [ ]:
validDomain = [4, 4, 4, 636, 636, 636]
block = [32, 2, 16]

totalGrid = [validDomain[i+3] // block[i] + 1 for i in range(3)]

print(totalGrid)

concurrentGrid = [totalGrid[0], totalGrid[1] // totalGrid[0], 1]

print(concurrentGrid)

In [ ]:
t = time.process_time()


loadExprs = {"A" : []}
for i in range(1, 5):
    loadExprs["A"].append("3 + tidx + tidy*1000 + (tidz+{})*1000*1000".format(i))
    loadExprs["A"].append("3 + tidx + (tidy+{})*1000 + tidz*1000*1000".format(i))
    loadExprs["A"].append("3 + (tidx+{}) + tidy*1000 + tidz*1000*1000".format(i))
    loadExprs["A"].append("3 + tidx + tidy*1000 + (tidz-{})*1000*1000".format(i))
    loadExprs["A"].append("3 + tidx + (tidy-{})*1000 + tidz*1000*1000".format(i))
    loadExprs["A"].append("3+ (tidx-{}) + tidy*1000 + tidz*1000*1000".format(i))

if not validDomain is None:
    validDomainSet = isl.BasicSet(
                "{{[x,y,z] : {0} <= x < {3} and {1} <= y < {4} and {2} <= z < {5} }}".format(
                    *validDomain
            )
        )
        # print("{{[x,y,z] : {0} <= x < {3} and {1} <= y < {4} and {2} <= z < {5} }}".format(
        #        *validDomain))

else:
    validDomainSet = isl.BasicSet("{[x, y, z]}")

print("Valid Domain: " + str(validDomainSet))
print()

metaGrid = tuple((totalGrid[i] - 1) // concurrentGrid[i] + 1 for i in range(3))

metaGridPosition = (
         metaGrid[0] // 2,
            metaGrid[1] // 2,
            metaGrid[2] // 2,
    )

metaGridId = (
        metaGridPosition[0]
        + metaGridPosition[1] * metaGrid[0]
        + metaGridPosition[2] * metaGrid[0] * metaGrid[1]
    )

prevMetaGridPosition = (
        (metaGridId - 1) % metaGrid[0],
        (metaGridId - 1) // metaGrid[0] % metaGrid[1],
        (metaGridId - 1) // (metaGrid[0] * metaGrid[1]),
    )

prev2MetaGridPosition = (
        (metaGridId - 2) % metaGrid[0],
        (metaGridId - 2) // metaGrid[0] % metaGrid[1],
        (metaGridId - 2) // (metaGrid[0] * metaGrid[1]),
    )

if (
        prev2MetaGridPosition[0] == prevMetaGridPosition[0]
        or prev2MetaGridPosition[1] == prevMetaGridPosition[1]
    ):
    prev2MetaGridPosition = prevMetaGridPosition

print("ISL Meta Grid:" + str(metaGrid))


concurrentThreads = tuple([block[i] * concurrentGrid[i] for i in range(3)])


cellCount = 0
Vold = 0
Vnew = 0
Voverlap = 0

for field in loadExprs:
    accesses = None
    first = True
    for access in loadExprs[field]:
        mapstring = (
                "{{[tidx, tidy, tidz] -> {}[idx] : idx = floor(({})/4)}}".format(
                    str(field), str(access)
                )
            )
        
        accessMap = isl.BasicMap(mapstring)
        if first:
            print(mapstring)
            print(accessMap)
            print()
            first = False
        
        if not accesses is None:
            accesses = accesses.union(accessMap)
        else:
            accesses = accessMap
    print()
    print(accesses)
    print()
    addresses = None
    prevAddresses = None
    cellCount = 0
    for tidz in range(0, block[2]):
        tidzSliceString = " {{[tidx, tidy, tidz] : {0} <= tidx < {3} and {1} <= tidy < {4} and {2} <= tidz < {5}}}".format(
                *(metaGridPosition[i] * concurrentThreads[i] for i in range(2)),
                metaGridPosition[2] * concurrentThreads[2] + tidz,
                *((metaGridPosition[i] + 1) * concurrentThreads[i] for i in range(2)),
                metaGridPosition[2] * concurrentThreads[2] + tidz + 1,
            )

        prevTidzSliceString = " {{[tidx, tidy, tidz] : {0} <= tidx < {3} and {1} <= tidy < {4} and {2} <= tidz < {5}}}".format(
                *(prevMetaGridPosition[i] * concurrentThreads[i] for i in range(2)),
                prevMetaGridPosition[2] * concurrentThreads[2] + tidz,
                *(
                    (prevMetaGridPosition[i] + 1) * concurrentThreads[i]
                    for i in range(2)
                ),
                prevMetaGridPosition[2] * concurrentThreads[2] + tidz + 1,
            )

        prev2TidzSliceString = " {{[tidx, tidy, tidz] : {0} <= tidx < {3} and {1} <= tidy < {4} and {2} <= tidz < {5}}}".format(
                *(prev2MetaGridPosition[i] * concurrentThreads[i] for i in range(2)),
                prev2MetaGridPosition[2] * concurrentThreads[2] + tidz,
                *(
                    (prev2MetaGridPosition[i] + 1) * concurrentThreads[i]
                    for i in range(2)
                ),
                prev2MetaGridPosition[2] * concurrentThreads[2] + tidz + 1,
            )
        tidzSliceDomain = isl.BasicSet(tidzSliceString).intersect(validDomainSet)
        
        
        prevTidzSliceDomain = isl.BasicSet(prevTidzSliceString).intersect(
                validDomainSet
            )

        prev2TidzSliceDomain = isl.BasicSet(prev2TidzSliceString).intersect(
                validDomainSet
            )
        #print(tidzSliceDomain.apply(accesses))
        #print()
        
        if not addresses is None:
            addresses = addresses.union(tidzSliceDomain.apply(accesses))
        else:
            addresses = tidzSliceDomain.apply(accesses)

        if not prevAddresses is None:
            prevAddresses = prevAddresses.union(prevTidzSliceDomain.apply(accesses))
            prevAddresses = prevAddresses.union(
                    prev2TidzSliceDomain.apply(accesses)
                )
        else:
            prevAddresses = prevTidzSliceDomain.apply(accesses)
            prevAddresses = prevAddresses.union(
                    prev2TidzSliceDomain.apply(accesses)
                )
        cellCount += tidzSliceDomain.count_val().to_python()
    
    Vnew += addresses.count_val().to_python()
    Vold += prevAddresses.count_val().to_python()
    Voverlap += addresses.intersect(prevAddresses).count_val().to_python()

cellCount = max(1, cellCount)  # avoid division by zero

print(
        Vnew * 32 / cellCount,
        " B/LUP\n",
        Vold * 32 / cellCount,
         " B/LUP\n",
        Voverlap * 32 / cellCount,
         " B/LUP\n",
        cellCount,
         " valid cells\n",
    )
elapsed_time = time.process_time() - t
print()
print("Time : " + str(elapsed_time) + " s")


In [ ]:
t = time.process_time()


loadExprs = {"A" : []}
for i in range(1, 5):
    loadExprs["A"].append( ("3 + tidx", "tidy", "tidz+{}".format(i)))
    loadExprs["A"].append( ("3 + tidx", "tidy", "tidz-{}".format(i)))
    loadExprs["A"].append( ("3 + tidx", "tidy+{}".format(i), "tidz"))
    loadExprs["A"].append( ("3 + tidx", "tidy-{}".format(i), "tidz"))
    loadExprs["A"].append( ("3 + tidx+{}".format(i), "tidy", "tidz"))
    loadExprs["A"].append( ("3 + tidx-{}".format(i), "tidy", "tidz"))
    
if not validDomain is None:
    validDomainSet = isl.BasicSet(
                "{{[x,y,z] : {0} <= x < {3} and {1} <= y < {4} and {2} <= z < {5} }}".format(
                    *validDomain
            )
        )
else:
    validDomainSet = isl.BasicSet("{[x, y, z]}")
    
    
metaGrid = tuple((totalGrid[i] - 1) // concurrentGrid[i] + 1 for i in range(3))

metaGridPosition = (
         metaGrid[0] // 2,
            metaGrid[1] // 2,
            metaGrid[2] // 2,
    )

metaGridId = (
        metaGridPosition[0]
        + metaGridPosition[1] * metaGrid[0]
        + metaGridPosition[2] * metaGrid[0] * metaGrid[1]
    )

prevMetaGridPosition = (
        (metaGridId - 1) % metaGrid[0],
        (metaGridId - 1) // metaGrid[0] % metaGrid[1],
        (metaGridId - 1) // (metaGrid[0] * metaGrid[1]),
    )

prev2MetaGridPosition = (
        (metaGridId - 2) % metaGrid[0],
        (metaGridId - 2) // metaGrid[0] % metaGrid[1],
        (metaGridId - 2) // (metaGrid[0] * metaGrid[1]),
    )

if (
        prev2MetaGridPosition[0] == prevMetaGridPosition[0]
        or prev2MetaGridPosition[1] == prevMetaGridPosition[1]
    ):
    prev2MetaGridPosition = prevMetaGridPosition

print("ISL Meta Grid:" + str(metaGrid))


concurrentThreads = tuple([block[i] * concurrentGrid[i] for i in range(3)])


cellCount = 0
Vold = 0
Vnew = 0
Voverlap = 0

for field in loadExprs:
    accesses = None
    first = True
    for access in loadExprs[field]:
        mapstring = (
                "{{[tidx, tidy, tidz] -> {0}[ax, ay, az] :   ax = floor(({1})/4) and ay = {2} and az = {3} }}".format(
                    str(field), *[str(a) for a in access]
                )
            )     
        
        accessMap = isl.BasicMap(mapstring)
        if first:
            print("Field Acess String: " + str( mapstring))
            print("Field Access Mapping: " + str(accessMap))
            print()
            first = False
            
        if not accesses is None:
            accesses = accesses.union(accessMap)
        else:
            accesses = accessMap
            
    
    print("Set of all field access mappings:")
    print(accesses)
    print()
        
    addresses = None
    prevAddresses = None
    cellCount = 0
    

    tidString = " {{[tidx, tidy, tidz] : {0} <= tidx < {3} and {1} <= tidy < {4} and {2} <= tidz < {5}}}".format(
                *(metaGridPosition[i] * concurrentThreads[i] for i in range(3)),
                *((metaGridPosition[i] + 1) * concurrentThreads[i] for i in range(3)))

    print("thread block set: " + str(tidString))
    
    prevTidString = " {{[tidx, tidy, tidz] : {0} <= tidx < {3} and {1} <= tidy < {4} and {2} <= tidz < {5}}}".format(
                *((prevMetaGridPosition[i] ) * concurrentThreads[i] for i in range(3)),
                *((prevMetaGridPosition[i] + 1) * concurrentThreads[i] for i in range(3)))
        
    
    prev2TidString = " {{[tidx, tidy, tidz] : {0} <= tidx < {3} and {1} <= tidy < {4} and {2} <= tidz < {5}}}".format(
                *((prev2MetaGridPosition[i] )* concurrentThreads[i] for i in range(3)),
                *((prev2MetaGridPosition[i] + 1) * concurrentThreads[i] for i in range(3)))

 
    tidDomain = isl.BasicSet(tidString).intersect(validDomainSet)
    print("threads in Domain: " + str(tidDomain))
    print()
    
    prevTidDomain = isl.BasicSet(prevTidString).intersect(validDomainSet)
    prev2TidDomain = isl.BasicSet(prev2TidString).intersect(validDomainSet)
        

        
    addresses = tidDomain.apply(accesses)
    print("Addresses: " + str(addresses))
    
    prevAddresses = prevTidDomain.apply(accesses)
    
    prev2Addresses = prevAddresses.union(prev2TidDomain.apply(accesses))
    
    elapsed_time = time.process_time() - t
    print(elapsed_time)
       
    cellCount += tidDomain.count_val().to_python()   
    Vnew += addresses.count_val().to_python()
    Vold += prevAddresses.count_val().to_python()
    Voverlap += addresses.intersect(prevAddresses).count_val().to_python()

cellCount = max(1, cellCount)  # avoid division by zero

print(
        Vnew * 32 / cellCount,
        " B/LUP\n",
        Vold * 32 / cellCount,
         " B/LUP\n",
        Voverlap * 32 / cellCount,
         " B/LUP\n",
        cellCount,
         " valid cells\n"
    )
elapsed_time = time.process_time() - t
print()
print("Time : " + str(elapsed_time) + " s")


In [ ]:
loadExprs = {"A" : [], "B" : []}
for i in range(1, 5):
    loadExprs["A"].append( ("3 + tidx", "tidy", "tidz+{}".format(i)))
    loadExprs["A"].append( ("3 + tidx", "tidy", "tidz-{}".format(i)))
    loadExprs["A"].append( ("3 + tidx", "tidy+{}".format(i), "tidz"))
    loadExprs["A"].append( ("3 + tidx", "tidy-{}".format(i), "tidz"))
    loadExprs["A"].append( ("3 + tidx+{}".format(i), "tidy", "tidz"))
    loadExprs["A"].append( ("3 + tidx-{}".format(i), "tidy", "tidz"))
loadExprs["B"].append( ("tidx", "tidy", "tidz") )


def getWaveMem(loadExprs, device, block, validDomain, concurrentBlockCount):
    def getISLVolumes( concurrentBlockCount ):
        t = time.process_time()


        if not validDomain is None:
            validDomainSet = isl.BasicSet(
                        "{{[x,y,z] : {0} <= x < {3} and {1} <= y < {4} and {2} <= z < {5} }}".format(
                            *validDomain
                    )
                )
        else:
            validDomainSet = isl.BasicSet("{[x, y, z]}")


        gridBlockCount = totalGrid[0] * totalGrid[1] * totalGrid[2]
        waveBlockCount = concurrentBlockCount

        currentWaveBlockSet = isl.BasicSet(
                        "{{[blockId] : {0} <= blockId < {1} and blockId < {2}}}".format( gridBlockCount // 2 ,
                                                                                         gridBlockCount // 2 + waveBlockCount,
                                                                                         gridBlockCount))

        blockIdToIdxStr = (
                        "{{[blockId] -> [blockIdx, blockIdy, blockIdz] : blockIdx = blockId mod {0}"
                        " and blockIdy = floor(blockId / {0}) mod {1} and "
                        " blockIdz = floor(blockId / {2}) }}".format(
                            totalGrid[0], totalGrid[1], totalGrid[1]*totalGrid[0]
                        )
                    )     

        blockIdToIdxMap = isl.BasicMap(blockIdToIdxStr)


        blockToThreadIdxStr= (
                        "{{[blockIdx, blockIdy, blockIdz] -> [tidx, tidy, tidz] : "
                        "blockIdx * {0} <= tidx < (blockIdx+1) * {0} and "
                        "blockIdy * {1} <= tidy < (blockIdy+1) * {1} and "
                        "blockIdz * {2} <= tidz < (blockIdz+1) * {2} "
                        "}}".format(
                            *block
                        )
                    )     
        blockToThreadIdxMap = isl.BasicMap(blockToThreadIdxStr)

        print(currentWaveBlockSet.apply(blockIdToIdxMap))
        currentWaveThreadSet = currentWaveBlockSet.apply(blockIdToIdxMap).apply(blockToThreadIdxMap)
        print(currentWaveThreadSet)

        tidDomain = currentWaveThreadSet.intersect(validDomainSet)

        print("threads in Domain: " + str(tidDomain))
        print()


        print()
        print()
        cellCount = 0
        Vold = 0
        Vnew = 0
        Voverlap = 0

        for field in loadExprs:
            accesses = None
            first = True
            for access in loadExprs[field]:
                mapstring = (
                        "{{[tidx, tidy, tidz] -> {0}[ax, ay, az] :   ax = floor(({1})/4) and ay = {2} and az = {3} }}".format(
                            str(field), *[str(a) for a in access] ))     

                accessMap = isl.BasicMap(mapstring)
                if first:
                    first = False

                if not accesses is None:
                    accesses = accesses.union(accessMap)
                else:
                    accesses = accessMap


            addresses = None
            prevAddresses = None
            cellCount = 0


            addresses = tidDomain.apply(accesses)


            elapsed_time = time.process_time() - t
            cellCount += tidDomain.count_val().to_python()   
            Vnew += addresses.count_val().to_python()
            #Vold += prevAddresses.count_val().to_python()
            #Voverlap += addresses.intersect(prevAddresses).count_val().to_python()

        cellCount = max(1, cellCount)  # avoid division by zero

        print(
                Vnew * 32 / cellCount,
                " B/LUP\n",
                Vold * 32 / cellCount,
                 " B/LUP\n",
                Voverlap * 32 / cellCount,
                 " B/LUP\n",
                cellCount,
                 " valid cells\n"
            )

        elapsed_time = time.process_time() - t
        print()
        print("Time : " + str(elapsed_time) + " s")
        return Vnew, cellCount

    
    Vnew, cellcount = getISLVolumes(concurrentBlockCount)
    while( Vnew*32 < 20 * 1024 * 1024):

        concurrentBlockCount = int (concurrentBlockCount *(20*1024*1024 / (Vnew*32)) * 1.1)
        print(Vnew * 32 / 1024 / 1024)
        print()
        print(concurrentBlockCount)
        Vnew, cellcount = getISLVolumes(concurrentBlockCount)
    print(Vnew * 32 / 1024 / 1024)
    
getWaveMem(loadExprs, None, block, validDomain, 108*2)

### 